# GEPA Hands-On: Optimized Listwise Reranker

This notebook will illustrate how to use the **GEPA optimizer in DSPy to train a Listwise Reranker**!

We find a jump in Recall @ 1 from **32% to 45%** with a GEPA-optimized prompt. 

This is after a small-scale optimization run with 500 metric calls, running in about an hour and a half. We think it is very likely that we could find a larger gain with a longer optimization run, as indicated by the Pareto frontier.

### What are Rerankers?
Rerankers are given as input: (1) a user's query and (2) a list of candidate documents identified by a first stage ranking algorithm, such as Hybrid Search. 

Rerankers then predict a **new ranking** of the candidate documents.

There are two common forms of rerankers, cross encoders and listwise rerankers. Cross Encoders take as input each query and candidate document in isolation, whereas a Listwise Reranker takes the query and all candidate documents as input.

This notebook simplifies this task a bit, focusing on identifying the **most** relevant document, or **best match**, rather than a ranking of the entire input set.

### GEPA Optimizer

The primary focus of this notebook is to illustrate how you can use the GEPA optimizer to develop AI systems.

GEPA introducs several innovations for prompt optimization such as **Reflective Prompt Mutation**, **System-Aware Merge**, and **Pareto-Optimal Candidate Selection**.

For a conceptual explanation of GEPA, we hope you will find [this conceptual explanation video](https://www.youtube.com/watch?v=czy7hvXIImE) useful, or our [interview with Lakshya A. Agrawal](https://www.youtube.com/watch?v=fREQrxhBSk0). You can also find the [research publication introducing GEPA here](https://arxiv.org/pdf/2507.19457).

In this example, we will aim to help clarify these concepts further by sharing our experience using GEPA!

### There are 6 steps in this notebook:

1. Setup DSPy Program `BestMatchReranker`

2. Load Training Dataset (**EnronQA**)

3. Define Metric with Natural Language Feedback

4. Evaluate Unoptimized `BestMatchReranker`

5. Run GEPA Optimizer

6. Evaluate Optimized `BestMatchReranker`

## 1. Setup DSPy Program `BestMatchReranker`

In [ ]:
import os
from pydantic import BaseModel
from typing import Optional, List, Dict, Any

import dspy

### CONFIGURE ###

os.environ["OPENAI_API_KEY"] = "sk-proj-foobar"

### MODELS ###

class SearchResult(BaseModel):
    id: int
    content: str
    dataset_id: Optional[str]

class Source(BaseModel):
    object_id: str

class DSPyAgentRAGResponse(dspy.Prediction):
    def __init__(self, final_answer: str = "", sources: List[Source] = None, 
                 searches: Optional[List[str]] = None, aggregations: Optional[List] = None,
                 usage: Optional[Dict[str, Any]] = None, **kwargs):
        super().__init__(**kwargs)
        self.final_answer = final_answer
        self.sources = sources or []
        self.searches = searches
        self.aggregations = aggregations
        self.usage = usage or {}

### SIGNATURES ###

class BestMatchRanker(dspy.Signature):
    """Identify the single most relevant passage to the query.
    
    Your task is to analyze ALL passages simultaneously and identify the one passage 
    that is most relevant for answering the query.
    
    Instructions:
    1. Read the query carefully and understand the information need
    2. Evaluate each passage for:
       - Direct relevance to answering the query
       - Factual accuracy and completeness
       - Information quality and clarity
    3. Compare passages against each other (not just individually)
    4. Return the ID of the single most relevant passage
    
    CRITICAL: You must return exactly 1 passage ID - the best match.
    """
    
    query: str = dspy.InputField(
        desc="The user's question or information need"
    )
    search_results: list[SearchResult] = dspy.InputField(
        desc="List of passages to analyze. Each contains: id, content"
    )
    best_match_id: int = dspy.OutputField(
        desc="The ID of the single most relevant passage. Must match an ID from search_results."
    )

### DSPy Language Program ###

class BestMatchReranker(dspy.Module):
    def __init__(
        self,
        verbose: bool = False,
    ):
        # init LLM
        self.lm = dspy.LM("openai/gpt-4.1-mini", api_key=os.getenv("OPENAI_API_KEY"))
        dspy.configure(lm=self.lm, track_usage=True)

        self.verbose = verbose
        self.reranker = dspy.ChainOfThought(BestMatchRanker) # update to send rationale through to metric

    def forward(self, question: str, candidates: list[SearchResult]) -> DSPyAgentRAGResponse:
        # Perform reranking
        rerank_pred = self.reranker(
            query=question,
            search_results=candidates,
        )
        
        # Find the best match result based on the returned ID
        best_match_result = None
        for candidate in candidates:
            if candidate.id == rerank_pred.best_match_id:
                best_match_result = candidate
                break
        
        reranked_sources = [best_match_result] if best_match_result else []
        
        if self.verbose:
            print(f"\033[96mReranked: Returning {len(reranked_sources)} Sources!\033[0m")
            if best_match_result:
                # Find the original position of this result in candidates
                original_rank = candidates.index(best_match_result) + 1  # +1 for 1-based ranking
                print(f"Best match ID: {rerank_pred.best_match_id} (was rank {original_rank})")
        
        # Get usage from reranker
        usage = rerank_pred.get_lm_usage() or {}
        
        return DSPyAgentRAGResponse(
            final_answer="",
            sources=reranked_sources,
            searches=[question],
            aggregations=None,
            usage=usage,
        )
    
    async def aforward(self, question: str) -> DSPyAgentRAGResponse:
        pass

reranker = BestMatchReranker()

## 2. Load Dataset

The dataset we will use can be found on [Weaviate's HuggingFace Repo](https://huggingface.co/datasets/weaviate/hard-questions-enronqa).

It contains 138 questions where state-of-the-art pre-trained Rerankers were able to achieve Recall @ 5, but not Recall @ 1, using the EnronQA dataset.

This presents an interesting opportunity to see if LLM-based rerankers can get us that extra mile to close the gap between Recall @ 5 and Recall @ 1.



In [10]:
from datasets import load_dataset

all_samples = load_dataset("weaviate/hard-questions-enronqa")['train']

In [11]:
all_samples[0]

{'question': "According to Sarah Novosel's email, what is the timeframe for submitting responses to the CPUC's data requests?",
 'shortlisted_candidates': [{'content': "The passage directly addresses the query by providing the timeframe for submitting responses to the CPUC's data requests, indicating that responses are due on the Friday following the email date. It clearly reflects Sarah Novosel's communication about the deadline, fulfilling the query's intent. The only limitation is the lack of an explicit calendar date for the Friday deadline, but the context allows for a reasonable inference.",
   'dataset_id': '4001',
   'id': 36},
  {'content': "The passage directly answers the query by stating that responses to the CPUC's data requests are due on Friday, according to Sarah Novosel's email. It effectively provides the timeframe for submission, fulfilling the query's intent. However, it does not specify the exact calendar date for the Friday deadline, which could be a minor limitat

### Dataset Preprocessing

We will quickly add the `ground_truth_content` to our samples so that GEPA can use it in the metric feedback.

(Sorry this isn't already in the dataset on HuggingFace)

In [12]:
# convert these samples to `dspy.Example` objects

import dspy

all_samples_cleaned = []

for sample in all_samples:
    candidates = []
    for idx, c in enumerate(sample["shortlisted_candidates"]):
        new_candidate = SearchResult(
            id=idx,
            content=c["content"],
            dataset_id=c["dataset_id"]
        )
        candidates.append(new_candidate)
    
    # Find ground truth content by matching dataset_id
    ground_truth_content = None
    for ground_truth_id in sample["ground_truths"]:
        for c in sample["shortlisted_candidates"]:
            if c["dataset_id"] == str(ground_truth_id):
                ground_truth_content = c["content"]
                break
        if ground_truth_content:
            break

    ex = dspy.Example().with_inputs("question", "candidates")
    ex["question"] = sample["question"]
    ex["candidates"] = candidates
    ex["ground_truths"] = sample["ground_truths"]
    ex["ground_truth_content"] = ground_truth_content

    all_samples_cleaned.append(ex)

print(len(all_samples_cleaned))

138


In [ ]:
all_samples_cleaned[0]["ground_truth_content"]

"The passage directly answers the query by stating that responses to the CPUC's data requests are due on Friday, according to Sarah Novosel's email. It effectively provides the timeframe for submission, fulfilling the query's intent. However, it does not specify the exact calendar date for the Friday deadline, which could be a minor limitation for precise scheduling."

In [17]:
reranker.forward(**all_samples_cleaned[0].inputs())

2025/08/21 16:27:19 WARNING dspy.primitives.module: Calling module.forward(...) on BestMatchReranker directly is discouraged. Please use module(...) instead.


Prediction(
    final_answer='',
    sources=[SearchResult(id=2, content="The passage directly addresses the query by providing the specific deadlines for submitting responses to the CPUC's data requests as communicated in the email. It clearly states the final date for submitting information (October 4, 2000) and the earlier deadline for submitting claims of privilege or confidentiality (September 29, 2000). This makes the passage highly relevant and complete in answering the query about the timeframe.", dataset_id='2386')],
    searches=["According to Sarah Novosel's email, what is the timeframe for submitting responses to the CPUC's data requests?"],
    aggregations=None,
    usage={'openai/gpt-4.1-mini': {'completion_tokens': 146, 'prompt_tokens': 904, 'total_tokens': 1050, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0, 'text_tokens': None}, 'prompt_tokens_details': {'audio_tokens': 0, 'cach

In [18]:
reranker.lm.inspect_history()





[2025-08-21T16:27:21.911466]

System message:

Your input fields are:
1. `query` (str): The user's question or information need
2. `search_results` (list[SearchResult]): List of passages to analyze. Each contains: id, text, initial_rank, and hybrid_score
Your output fields are:
1. `reasoning` (str): 
2. `best_match_id` (int): The ID of the single most relevant passage. Must match an ID from search_results.
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## search_results ## ]]
{search_results}

[[ ## reasoning ## ]]
{reasoning}

[[ ## best_match_id ## ]]
{best_match_id}        # note: the value you produce must be a single int value

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Identify the single most relevant passage to the query.
        
        Your task is to analyze ALL passages simultaneously and identify the one passage 
        that is most relevant for answ

## 3. Metric with Feedback

One of the key innovations in GEPA is to provide the Prompt Proposer with more detailed information about input, output pairs. Rather than a scalar reward, such as "0.7", the metric sends back natural language feedback about what went wrong or right with this inference.

In our case, we will show the ground truth document that should have been ranked so that the Prompt Proposer can try to reverse engineer why the current instructions selected the document it did instead of the correct one.

In [19]:
# define the metric
from dspy import Example, Prediction

def recall_metric_with_feedback(
        example: Example, 
        prediction, 
        trace=None,
        pred_name=None,
        pred_trace=None
    ) -> Prediction:
        retrieved_id = prediction.sources[0].dataset_id # only reranking to 1 result for now
            
        ground_truth = str(example.ground_truths[0])
            
        if retrieved_id == ground_truth:
            return Prediction(
                score=1.0,
                feedback="Awesome! The system correctly predicted the top document."
            )
        else:
            predicted_content = prediction.sources[0].content
            ground_truth_content = example.ground_truth_content
            question = example.question
            return Prediction(
                score=0.0,
                feedback=f"Incorrect document selected for the query: {question}. The correct answer was: {ground_truth_content}. The system incorrectly predicted {predicted_content}."
            )

## 4. Run Unoptimized Eval

Evaluate our `BestMatchReranker` before optimizing it with GEPA!

In [21]:
trainset, testset = all_samples_cleaned[:100], all_samples_cleaned[100:]

In [22]:
# run the evaluator on the test set to understand the performance of the zero-shot listwise reranker
evaluator = dspy.Evaluate(
    devset=testset,
    metric=recall_metric_with_feedback, 
    num_threads=1,
    display_progress=True,
    max_errors=1,
    provide_traceback=True
)

dspy_evaluator_kwargs = {
    "num_threads": 5
}

evaluator(reranker, **dspy_evaluator_kwargs)

Average Metric: 12.00 / 38 (31.6%): 100%|██████████| 38/38 [00:20<00:00,  1.86it/s]

2025/08/21 16:27:53 INFO dspy.evaluate.evaluate: Average Metric: 12.0 / 38 (31.6%)


EvaluationResult(score=31.58, results=<list of 38 results>)

`Note:` We improve from 4/38 to 12/38 with summarized query <> candidate document relevances used for reranking, rather than the raw emails.

## 5. GEPA Optimization

As a quick reminder, GEPA is using a **Pareto-frontier** to sample candidates. 

Each candidate on the frontier is better than all the other candidates on at least 1 of your validation samples. 

This is critical to understanding how the optimizer works:

![pareto_frontier](./pareto-sampling.png)

Before we begin by constructing and running the GEPA optimizer. Here are a couple of tips we recommend for monitoring the optimization run.

1. Setup Weights & Biases Logging!

This is already built into the GEPA optimizer. It will log things that make it easy to keep track of your GEPA run such as the best score on your entire validation set, the Pareto frontier score, and the iteration of the training run, amongst others.


### Weights & Biases Logging

#### Best Overall Score
![agg_score](./wandb-gepa-2.png)

#### Pareto Frontier
![pareto](./wandb-gepa-1.png)

2. Check in with Gemini

We found it very helpful to copy and paste the output from GEPA and ask Gemini --

```
Can you analyze this prompt optimization run? How's it going?

{paste your GEPA output here}
```

### Code to setup the GEPA Optimizer Run

In [ ]:
# optimize!!

import dspy

import logging

# Simple setup for Jupyter
logging.basicConfig(level=logging.INFO, force=True)
logging.getLogger('dspy.teleprompt.gepa').setLevel(logging.INFO)
logging.getLogger('gepa').setLevel(logging.INFO)

# SILENCE the noisy HTTP loggers
logging.getLogger('httpx').setLevel(logging.WARNING)  # Only warnings and errors
logging.getLogger('openai').setLevel(logging.WARNING)
logging.getLogger('weaviate').setLevel(logging.WARNING)
logging.getLogger('httpcore').setLevel(logging.WARNING)

reflection_lm = dspy.LM(
    model="gpt-5",
    temperature=1.0,
    max_tokens=32_000
)

wandb_config = {
    "project": "gepa-best-match-ranker-run",
    "name": "run-500-calls-val25-merge10",
    "notes": "Increasing budget and merge invocations, reduce val set size.",
    "config": {
        "max_metric_calls": 500,
        "reflection_minibatch_size": 5,
        "max_merge_invocations": 10,
        "val_subset_size": 25,
        "train_size": 75
    }
}

optimizer = dspy.GEPA(
    metric=recall_metric_with_feedback,
    max_metric_calls=500,
    reflection_lm=reflection_lm,
    reflection_minibatch_size=5,
    use_merge=True,
    max_merge_invocations=10,
    num_threads=5,
    use_wandb=True,
    wandb_api_key=os.getenv("WANDB_API_KEY"),
    wandb_init_kwargs=wandb_config
)

train_samples=trainset[:75]
val_samples=trainset[75:]

optimized_reranker = optimizer.compile(
    reranker,
    trainset=train_samples,
    valset=val_samples
)

2025/08/21 16:44:19 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 500 metric calls of the program. This amounts to 5.00 full evals on the train+val set.
2025/08/21 16:44:19 INFO dspy.teleprompt.gepa.gepa: Using 25 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/jupyter_events/schema.py:6: DeprecationWarning: jsonschema.RefResolver is deprecated as of v4.18.0, in favor of the https://github.com/python-jsonschema/referencing library, which provides more compliant referencing behavior as well as more flexible APIs for customization. A future release will remove RefResolver. Please file a feature request (on referencing) if you are missing an API for the kind of customization you need.
  from jsonschema import FormatChecker, RefResolver, validators
/Library/Frameworks/Python.framework/Ver

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/main.py:453: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='[[ ## re...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/main.py:453: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content="[[ ## re...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpec

Average Metric: 0.00 / 5 (0.0%): 100%|██████████| 5/5 [00:04<00:00,  1.23it/s]

2025/08/21 16:44:37 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 5 (0.0%)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pydantic/main.py:453: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 6: Expected `Message` - serialized value may not be as expected [input_value=Message(content='```\nTas...: None}, annotations=[]), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...ider_specific_fields={}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
2025/08/21 16:45:17 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Proposed new text for reranker.predict: Task: Given a query and a list of SearchResult items (each with id, content, dataset_id), identify the single most relevant passage to answer the query and return exactly one passage ID (SearchResult.id).

Input format:
- query: A natural-language question asking for a specific fact (wh

...

2025/08/21 17:27:05 INFO dspy.evaluate.evaluate: Average Metric: 1.0 / 5 (20.0%)


2025/08/21 17:28:46 INFO dspy.teleprompt.gepa.gepa: Iteration 29: Proposed new text for reranker.predict: Task
Given a query and a list of passages (SearchResult objects), analyze ALL passages simultaneously and select the single passage that best answers the query. Return exactly one passage ID (the integer id field), and nothing else.

Input format
- query: a natural-language question (often attributed to a specific person/email or tied to a specific event/date/case).
- search_results: a list of SearchResult objects with:
  - id: integer identifier to return
  - content: a short description of what the passage asserts in relation to the query (often meta-summaries like “The passage directly addresses the query by stating …”)
  - dataset_id: string identifier (do not return this)

Output format
- Exactly one integer: the id of the single best passage. No extra text, no reasoning.

General approach
1) Parse the query carefully and extract every constraint:
   - Who/attribution: e.g., “

Average Metric: 0.00 / 5 (0.0%): 100%|██████████| 5/5 [00:02<00:00,  2.15it/s]

2025/08/21 17:28:51 INFO dspy.evaluate.evaluate: Average Metric: 0.0 / 5 (0.0%)


2025/08/21 17:30:35 INFO dspy.teleprompt.gepa.gepa: Iteration 30: Proposed new text for reranker.predict: You are given a query and a list of passages (search_results). Your job is to select the single passage that best answers the query and return exactly one passage ID (the SearchResult.id integer). Do not include any explanation—output only the integer ID.

Input format:
- query: a natural-language question that may include constraints such as the source (e.g., “According to James Steffes’ email”), the scope (“the issue at hand”), or the exact type of answer (entity, date/month, list of two items).
- search_results: a list of SearchResult objects with fields:
  - id: the integer identifier you must return
  - content: a short summary/assessment of the passage’s relevance
  - dataset_id: a source identifier (do not return this)

Core decision process (follow in order):
1) Parse the query and extract all constraints:
   - Source attribution: person and medium (e.g., “According to Sara

## 6. Save and Evaluate Optimized `BestMatchReranker`

In [25]:
# save optimized listwise reranker
optimized_reranker.save("gepa_optimized_best_match_reranker.json")

In [26]:
# evaluate optimized listwise reranker on the test set
dspy_evaluator_kwargs = {
    "num_threads": 5
}

evaluator(optimized_reranker, **dspy_evaluator_kwargs)

Average Metric: 12.00 / 19 (63.2%):  47%|████▋     | 18/38 [00:10<00:12,  1.61it/s]

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/re.py:247: ResourceWarning: unclosed <ssl.SSLSocket fd=156, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.0.0.233', 54140), raddr=('162.159.140.245', 443)>
  return _compile(pattern, flags).finditer(string)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/re.py:247: ResourceWarning: unclosed <ssl.SSLSocket fd=158, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.0.0.233', 54139), raddr=('162.159.140.245', 443)>
  return _compile(pattern, flags).finditer(string)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/re.py:247: ResourceWarning: unclosed <ssl.SSLSocket fd=162, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('10.0.0.233', 54143), raddr=('162.159.140.245', 443)>
  return _compile(pattern, flags).finditer(string)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/re.py:247: Resou

Average Metric: 17.00 / 38 (44.7%): 100%|██████████| 38/38 [00:21<00:00,  1.75it/s]

2025/08/21 17:36:19 INFO dspy.evaluate.evaluate: Average Metric: 17.0 / 38 (44.7%)


EvaluationResult(score=44.74, results=<list of 38 results>)

sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x30b0b5a80>


### Visualize Optimized Prompt

In [27]:
optimized_reranker.lm.inspect_history()





[2025-08-21T17:30:40.131122]

System message:

Your input fields are:
1. `query` (str): The user's question or information need
2. `search_results` (list[SearchResult]): List of passages to analyze. Each contains: id, text, initial_rank, and hybrid_score
Your output fields are:
1. `reasoning` (str): 
2. `best_match_id` (int): The ID of the single most relevant passage. Must match an ID from search_results.
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## search_results ## ]]
{search_results}

[[ ## reasoning ## ]]
{reasoning}

[[ ## best_match_id ## ]]
{best_match_id}        # note: the value you produce must be a single int value

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are given a query and a list of passages (search_results). Your job is to select the single passage that best answers the query and return exactly one passage ID (the SearchResult.id intege